# Second model: convolutions
## Data augmentation, learning rate decay, dropout.

In [1]:


import numpy as np
import matplotlib
matplotlib.use('TkAgg', warn = False)
from matplotlib import pyplot
import os
import time
from PIL import Image

from pandas import DataFrame
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from nolearn.lasagne import BatchIterator
import tensorflow as tf
print("done loading libraries")

ModuleNotFoundError: No module named 'nolearn'

## Loading data

Paths to datasets.

In [3]:
FROOT = os.getcwd() # Path to your project folder
FTRAIN = FROOT + '\data\\train.csv'
print(FTRAIN)
FTEST = FROOT + '\data\\test.csv'
print(FTEST)
#FLOOKUP = FROOT + '/data/IdLookupTable.csv'
Irows=640
Icols=480

D:\FYP\CNN\muct-master\allimger\data\train.csv
D:\FYP\CNN\muct-master\allimger\data\test.csv


In [4]:
def scaley(y,rows,cols):
	#loop for rows
	dr=cols/2
	for i in range(0,152,2):
		y[:,i]=y[:,i]/dr
	#loop for cols
	dc=rows/2
	for i in range(1,152,2):
		y[:,i]=y[:,i]/dc
	return y

def load(test = False, cols = None):
    """
    Loads the dataset.
    
    Parameters
    ----------
    test     : optional, defaults to `False`
               Flag indicating if we need to load from `FTEST` (`True`) or FTRAIN (`False`)
    cols     : optional, defaults to `None`
               A list of columns you're interested in. If specified only returns these columns.
    Returns
    -------
    A tuple of X and y, if `test` was set to `True` y contains `None`.    
    """
    
    fname = FTEST if test else FTRAIN
    df = read_csv(os.path.expanduser(fname))  # load pandas dataframe

    # The Image column has pixel values separated by space; convert
    # the values to numpy arrays:
    df['image'] = df['image'].apply(lambda im: np.fromstring(im, sep = ','))

    if cols:  # get a subset of columns
        df = df[list(cols) + ['image']]

    print(df.count())  # prints the number of values for each column
    df = df.dropna()  # drop all rows that have missing values in them

    X = np.vstack(df['image'].values) / 255.  # scale pixel values to [0, 1]
    X = X.astype(np.float32)

    if not test:  # only FTRAIN has any target columns
        y = df[df.columns[4:-1]].values
        y = scaley(y,640,480)   #(y - 48) / 48   scale target coordinates to [-1, 1]
        X, y = shuffle(X, y, random_state=42)  # shuffle train data
        y = y.astype(np.float32)
    else:
        y = None

    return X, y

In [5]:
def plot_sample(x, y, axis):
    """
    Plots a single sample image with keypoints on top.
    
    Parameters
    ----------
    x     : 
            Image data.
    y     : 
            Keypoints to plot.
    axis  :
            Plot over which to draw the sample.   
    """
    img = x.reshape(Irows, Icols)
    axis.imshow(img, cmap='gray')
    axis.scatter(y[0::2] * 240 + 240, y[1::2] * 320 + 320, marker='x', s=10)

In [6]:
num_channels = 1 # grayscale
image_size = 96

def load2d(test = False, cols = None):
    X, y = load(test = test)
    X = X.reshape(-1, Irows, Icols, num_channels)
    return X, y

Load the training dataset.

In [7]:
#X, y = load2d()
print("done loading traing dataset")

done loading traing dataset


Split the initial training dataset into training, validation and testing datasets.

In [8]:
#x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
#x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test, test_size = 0.5)
print("done spliting")

done spliting


This is our data augmentation routine. It randomly flips a defined portion of dataset horizontally.

In [9]:

class FlipBatchIterator(BatchIterator):
    """
    Batch iterator that randomly flips a defined portion of dataset horizontally.
    """
    
    flip_indices = [
        (0,28),(1,29),(2,26),(3,27),(4,24),
        (5,25),(6,22),(7,23),(8,20),(9,21),
        (10,18),(11,19),(12,16),(13,17),(30,42),
        (31,43),(32,44),(33,45),(34,46),(35,47),
        (36,48),(37,49),(38,50),(39,51),(40,52),
        (41,53),(54,64),(55,65),(56,66),(57,67),
        (58,68),(59,69),(60,70),(61,71),(62,72),
        (63,73),(74,90),(75,91),(76,88),(77,89),
        (78,86),(79,87),(80,84),(81,85),(92,94),
        (93,95),(96,108),(97,109),(98,106),(99,107),
        (100,104),(101,105),(110,118),(111,119),
        (112,116),(113,117),(120,124),(121,125),
        (126,130),(127,131),(136,144),(137,145),
        (138,146),(139,147),(140,148),(141,149),
        (142,150),(143,151)
        ]

    def transform(self, Xb, yb):
        Xb, yb = super(FlipBatchIterator, self).transform(Xb, yb)

        # Flip half of the images in this batch at random:
        bs = Xb.shape[0]
        indices = np.random.choice(bs, int(bs / 2), replace=False)
        Xb[indices] = Xb[indices, :, ::-1, :]

        if yb is not None:
            # Horizontal flip of all x coordinates:
            yb[indices, ::2] = yb[indices, ::2] * -1

            # Swap places, e.g. left_eye_center_x -> right_eye_center_x
            for a, b in self.flip_indices:
                yb[indices, a], yb[indices, b] = (
                    yb[indices, b], yb[indices, a])

        return Xb, yb

Try flipping a sample batch of images and plot it to verify it's all good. What you should see is half of images flipped.

In [10]:
#batch_iterator = FlipBatchIterator(batch_size = 10)
#for x_batch, y_batch in batch_iterator(X.copy(), y.copy()):
#    for i in range(1):        
#        # plot two images:
#        fig = pyplot.figure(figsize=(6, 3))
#        ax = fig.add_subplot(1, 2, 1, xticks=[], yticks=[])
#        plot_sample(X[i], y[i], ax)
#        ax = fig.add_subplot(1, 2, 2, xticks=[], yticks=[])
#        plot_sample(x_batch[i], y_batch[i], ax)
#        pyplot.show()
#    break
print("done line 10")

done line 10


## Parameters

In [11]:
# Predefined parameters
num_keypoints = 152
batch_size = 36
num_epochs = 101

data_augmentation = True
learning_rate_decay = True
momentum_increase = True
dropout = True

# We will encode model settings in its name: architecture, batch size, number of epochs and optimisations applied.
model_name = "MUCTmodel"+"_3con_2fc_b" + str(batch_size) + "_e" + str(num_epochs - 1)
if data_augmentation:
    model_name += "_aug"
if learning_rate_decay:
    model_name += "_lrdec"
if momentum_increase:
    model_name += "_mominc"
if dropout:
    model_name += "_dr"
    
model_variable_scope = model_name
root_location = FROOT + "/models/"
model_path = root_location + model_name + "/model.ckpt"
train_history_path = root_location + model_name + "/train_history"

os.makedirs(root_location + model_name + "/", exist_ok = True)

## Routines

Performs a single fully connected layer pass, e.g. returns `input * weights + bias`:

In [12]:
def fully_connected(input, size):
    """
    Creates a fully connected TensorFlow layer.
    
    Parameters
    ----------
    input  : 
            Input tensor for calculating layer shape.
    size   : 
            Layer size, e.g. number of units.
               
    Returns
    -------
    A graph variable calculating single fully connected layer.
    """
    weights = tf.get_variable( 'weights', 
        shape = [input.get_shape()[1], size],
        initializer = tf.contrib.layers.xavier_initializer()
      )
    biases = tf.get_variable( 'biases',
        shape = [size],
        initializer=tf.constant_initializer(0.0)
      )
    return tf.matmul(input, weights) + biases

def fully_connected_relu(input, size):
    """
    Creates a fully connected TensorFlow layer with ReLU non-linearity applied.
    """
    return tf.nn.relu(fully_connected(input, size))

Routine for a single convolution layer pass.

In [13]:
def conv_relu(input, kernel_size, depth):
    """
    Creates a convolutional TensorFlow layer followed by a ReLU.
    
    Parameters
    ----------
    input         : 
                    Input tensor for calculating layer shape.
    kernel_rsize   : 
                    Kernel row size.
    kernel_csize   : 
                    Kernel column size.
    depth         : 
                    Layer depth, e.g. number of units.
               
    Returns
    -------
    A graph variable calculating convolutional layer with applied ReLU.
    """
    weights = tf.get_variable( 'weights', 
        shape = [kernel_size, kernel_size, input.get_shape()[3], depth],
        initializer = tf.contrib.layers.xavier_initializer()
      )
    biases = tf.get_variable( 'biases',
        shape = [depth],
        initializer=tf.constant_initializer(0.0)
      )
    conv = tf.nn.conv2d(input, weights,
        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

Routine for a pooling layer.

In [14]:
def pool(input, size):
    """
    Performs max pooling.
    
    Parameters
    ----------
    input  : 
            Input tensor.
    rsize   : 
            Pooling kernel row size.
    csize   : 
            Pooling kernel  column size.
               
    Returns
    -------
    A graph variable calculating single max pooling layer.
    """
    return tf.nn.max_pool(
        input, 
        ksize=[1, size, size, 1], 
        strides=[1, size, size, 1], 
        padding='SAME'
    )

Routine that performs entire model pass, e.g. returns model prediction for given input with current model (3 convolution layers with 2 fully connected layers):

In [15]:
def model_pass(input, training):
    """
    Performs a whole model pass.
    
    Parameters
    ----------
    input     : 
                Input tensor to be passed through the model.
    training  : 
                Tensorflow flag indicating if we are training or evaluating our model 
                (so that we know if we should apply dropout).
               
    Returns
    -------
    Model prediction.
    """
    # Convolutional layers
    with tf.variable_scope('conv1'):
        conv1 = conv_relu(input, kernel_size = 3, depth = 32) 
        pool1 = pool(conv1, size = 2)
        # Apply dropout if needed
        pool1 = tf.cond(training, lambda: tf.nn.dropout(pool1, keep_prob = 0.9 if dropout else 1.0), lambda: pool1)
    with tf.variable_scope('conv2'):
        conv2 = conv_relu(pool1, kernel_size = 2, depth = 64)
        pool2 = pool(conv2, size = 2)
        # Apply dropout if needed
        pool2 = tf.cond(training, lambda: tf.nn.dropout(pool2, keep_prob = 0.8 if dropout else 1.0), lambda: pool2)
    with tf.variable_scope('conv3'):
        conv3 = conv_relu(pool2, kernel_size = 2, depth = 128)
        pool3 = pool(conv3, size = 2)
        # Apply dropout if needed
        pool3 = tf.cond(training, lambda: tf.nn.dropout(pool3, keep_prob = 0.7 if dropout else 1.0), lambda: pool3)
    
    # Flatten convolutional layers output
    shape = pool3.get_shape().as_list()
    flattened = tf.reshape(pool3, [-1, shape[1] * shape[2] * shape[3]])
    
    # Fully connected layers
    with tf.variable_scope('fc4'):
        fc4 = fully_connected_relu(flattened, size = 1000)
        # Apply dropout if needed
        fc4 = tf.cond(training, lambda: tf.nn.dropout(fc4, keep_prob = 0.5 if dropout else 1.0), lambda: fc4)
    with tf.variable_scope('fc5'):
        fc5 = fully_connected_relu(fc4, size = 1000)
    with tf.variable_scope('out'):
        prediction = fully_connected(fc5, size = num_keypoints)
    return prediction

Calculates loss based on model predictions.

In [16]:
def calc_loss(predictions, labels):
    """
    Calculates loss with NumPy.
    
    Parameters
    ----------
    predictions : ndarray 
                  Predictions.
    labels      : ndarray
                  Actual values.
               
    Returns
    -------
    Squared mean error for given predictions.
    """
    return np.mean(np.square(predictions - labels))

Calculates time since `start` and formats as a string.

In [17]:
def get_time_hhmmss(start):
    """
    Calculates time since `start` and formats as a string.
    
    Parameters
    ----------
    start :  
            Time starting point.
               
    Returns
    -------
    Nicely formatted time difference between now and `start`.
    """
    end = time.time()
    m, s = divmod(end - start, 60)
    h, m = divmod(m, 60)
    time_str = "%02d:%02d:%02d" % (h, m, s)
    return time_str

## Training

In [18]:
graph = tf.Graph()
print("start")
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
    tf_x_batch = tf.placeholder(tf.float32, shape = (None, Irows, Icols, num_channels))
    tf_y_batch = tf.placeholder(tf.float32, shape = (None, num_keypoints))
    is_training = tf.placeholder(tf.bool)

    current_epoch = tf.Variable(0)  # count the number of epochs

    # Model parameters.
    
    # Calculate learning rate decay if needed.
    if learning_rate_decay:
        learning_rate = tf.train.exponential_decay(0.03, current_epoch, decay_steps=num_epochs, decay_rate=0.03)
    else:
        learning_rate = 0.01
        
    # Calculate increasing momentum if needed.
    if momentum_increase:
        m_min = 0.9
        m_max = 0.99
        momentum = m_min + (m_max - m_min) * (current_epoch / num_epochs) 
    else:
        momentum = 0.9
    
    # Training computation.
    with tf.variable_scope(model_variable_scope):
        predictions = model_pass(tf_x_batch, is_training)
    
    loss = tf.reduce_mean(tf.square(predictions - tf_y_batch))

    # Optimizer.
    optimizer = tf.train.MomentumOptimizer(
        learning_rate = learning_rate, 
        momentum = momentum, 
        use_nesterov = True
    ).minimize(loss)
    
    
print("done")

start
Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
done


In [19]:
def get_predictions_in_batches(X, session):
    """
    Calculates predictions in batches of 128 examples at a time, using `session`'s calculation graph.
    
    Parameters
    ----------
    X       : ndarray
              Dataset to get predictions for.
    session :
              TensorFlow session to be used for predicting. Is expected to have a `predictions` var 
              in the graph along with a `tf_x_batch` placeholder for incoming data.
               
    Returns
    -------
    N-dimensional array of predictions.
    """
    p = []
    batch_iterator = BatchIterator(batch_size = 128)
    for x_batch, _ in batch_iterator(X):
        [p_batch] = session.run([predictions], feed_dict = {
                tf_x_batch : x_batch,
                is_training : False
            }
        )
        p.extend(p_batch)
    return p

In [21]:
start = time.time()
every_epoch_to_log = 5
""""
import pyrealsense2 as rs

bag_file_path="test.bag"

pipeline = rs.pipeline()
config = rs.config()
rs.config.enable_device_from_file(config, bag_file_path)
config.enable_stream(rs.stream.depth)
config.enable_stream(rs.stream.color)

profile = pipeline.start(config)
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()

align_to = rs.stream.color
align = rs.align(align_to)
try:
    while True:
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        aligned_depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()

        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        num_rows = depth_image.shape[0]
        num_cols = depth_image.shape[1]

        depth_intrin = aligned_depth_frame.profile.as_video_stream_profile().intrinsics

        for r in range(0, num_rows, 5):
            for c in range(0, num_cols, 5):
                depth = aligned_depth_frame.get_distance(c, r)
                depth_point_in_meters_camera_coords = rs.rs2_deproject_pixel_to_point(depth_intrin, [c, r], depth)
                #some other stuff I do here with this depth point, such as projecting it to world coordinates etc.

finally:
    pipeline.stop()

"""

    
    

model_path2=FROOT+"\muctdata\models\MUCTmodel_3con_2fc_b36_e100_aug_lrdec_mominc_dr\model.ckpt"
Image_loc="image1_3.jpg"
img=Image.open(Image_loc).convert('LA')
img.show()
img_arr=np.array(img)[:,:,0]
img_arrnormal=img_arr/255
X = img_arrnormal.astype(np.float32)
XR=X.reshape(-1,Irows,Icols,1)
pred=[]

with tf.Session(graph = graph) as sess:
    
    saver = tf.train.Saver()
    saver.restore(sess,model_path2)
    print("model restored")
    pred=sess.run([predictions], feed_dict = {
                tf_x_batch : XR,
                is_training : False
            }
                    )
print("prediction are: " + str(pred))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from D:\FYP\CNN\muct-master\allimger\muctdata\models\MUCTmodel_3con_2fc_b36_e100_aug_lrdec_mominc_dr\model.ckpt
model restored
prediction are: [array([[-4.09757018e-01,  9.70426947e-03, -4.20393825e-01,
         1.00986220e-01, -4.03695345e-01,  1.92281395e-01,
        -3.94951820e-01,  2.89573044e-01, -3.33921462e-01,
         3.90932828e-01, -2.47292325e-01,  4.63322341e-01,
        -1.32102624e-01,  4.98232961e-01, -2.77678668e-03,
         5.16548872e-01,  1.22197241e-01,  4.99967843e-01,
         2.39044249e-01,  4.56127167e-01,  3.28049630e-01,
         3.96210134e-01,  3.85517359e-01,  2.95656800e-01,
         4.05359030e-01,  1.99052751e-01,  4.10661906e-01,
         1.06420457e-01,  4.05348420e-01,  9.70695913e-03,
         3.07913005e-01, -4.51300740e-02,  2.36076683e-01,
        -1.00042708e-01,  1.64960876e-01, -1.07555471e-01,
         8.51064250e-02,

## Evaluation

In [22]:
def scaleyinv(y,rows,cols):
    #loop for rows
    dr=cols/2
    for i in range(0,152,2):
        y[i]=(y[i]*dr)+dr
    #loop for cols
    dc=rows/2
    for i in range(1,152,2):
        y[i]=(y[i]*dc)+dc
    return y


In [23]:
npred=np.array(pred[0][0])
print(npred.shape)
print(npred)
#print(str(pred))


(152,)
[-4.09757018e-01  9.70426947e-03 -4.20393825e-01  1.00986220e-01
 -4.03695345e-01  1.92281395e-01 -3.94951820e-01  2.89573044e-01
 -3.33921462e-01  3.90932828e-01 -2.47292325e-01  4.63322341e-01
 -1.32102624e-01  4.98232961e-01 -2.77678668e-03  5.16548872e-01
  1.22197241e-01  4.99967843e-01  2.39044249e-01  4.56127167e-01
  3.28049630e-01  3.96210134e-01  3.85517359e-01  2.95656800e-01
  4.05359030e-01  1.99052751e-01  4.10661906e-01  1.06420457e-01
  4.05348420e-01  9.70695913e-03  3.07913005e-01 -4.51300740e-02
  2.36076683e-01 -1.00042708e-01  1.64960876e-01 -1.07555471e-01
  8.51064250e-02 -6.33961558e-02  1.67819902e-01 -7.34384358e-02
  2.39568233e-01 -7.12789744e-02 -3.32991898e-01 -5.75358644e-02
 -2.65245497e-01 -1.12159632e-01 -1.78951755e-01 -9.77146626e-02
 -9.47979391e-02 -7.50247240e-02 -1.68786347e-01 -6.14172891e-02
 -2.60976851e-01 -6.40557706e-02 -2.69603133e-01  1.06885955e-02
 -2.01534361e-01 -3.11491489e-02 -1.36702031e-01 -7.06210732e-03
 -1.96867496e-01  

In [24]:
oupred=scaleyinv(npred,Irows,Icols)
print(oupred)

[141.65831 323.10538 139.10548 352.31558 143.11311 381.53006 145.21156
 412.66336 159.85886 445.0985  180.64984 468.26315 208.29536 479.43454
 239.33357 485.29565 269.32733 479.98972 297.3706  465.9607  318.7319
 446.78723 332.52417 414.61017 337.28616 383.69687 338.55887 354.05453
 337.28363 323.10623 313.8991  305.55838 296.65842 287.98633 279.5906
 285.58224 260.42554 299.71323 280.27676 296.4997  297.49637 297.19073
 160.08194 301.58853 176.34108 284.10892 197.05157 288.73132 217.24849
 295.9921  199.49127 300.34647 177.36555 299.50217 175.29524 323.42035
 191.63176 310.0323  207.19151 317.7401  192.7518  327.04312 190.93027
 318.83853 302.00443 322.0567  284.51297 312.5361  268.89285 325.2046
 284.03198 326.43732 282.22083 319.4238  227.54012 319.33313 224.05124
 347.938   208.3053  366.87076 215.29884 375.16397 238.80269 380.51672
 260.32578 376.9832  269.7714  371.2343  251.79834 344.35324 248.82443
 317.25195 223.94894 372.12994 253.06474 373.4489  199.12306 413.47202
 219.4547

In [25]:
oupred.shape

(152,)

In [26]:
nop=np.floor(oupred)

In [27]:
print(nop)

[141. 323. 139. 352. 143. 381. 145. 412. 159. 445. 180. 468. 208. 479.
 239. 485. 269. 479. 297. 465. 318. 446. 332. 414. 337. 383. 338. 354.
 337. 323. 313. 305. 296. 287. 279. 285. 260. 299. 280. 296. 297. 297.
 160. 301. 176. 284. 197. 288. 217. 295. 199. 300. 177. 299. 175. 323.
 191. 310. 207. 317. 192. 327. 190. 318. 302. 322. 284. 312. 268. 325.
 284. 326. 282. 319. 227. 319. 224. 347. 208. 366. 215. 375. 238. 380.
 260. 376. 269. 371. 251. 344. 248. 317. 223. 372. 253. 373. 199. 413.
 219. 399. 230. 398. 239. 400. 247. 396. 259. 399. 279. 413. 266. 423.
 254. 428. 239. 428. 224. 427. 210. 424. 222. 416. 239. 417. 255. 417.
 254. 408. 240. 406. 222. 408. 238. 409. 237. 362. 182. 316. 199. 318.
 201. 324. 183. 323. 290. 316. 276. 316. 274. 322. 292. 324.]


In [28]:
nop1=np.reshape(nop,(76,2))
print(nop1)

[[141. 323.]
 [139. 352.]
 [143. 381.]
 [145. 412.]
 [159. 445.]
 [180. 468.]
 [208. 479.]
 [239. 485.]
 [269. 479.]
 [297. 465.]
 [318. 446.]
 [332. 414.]
 [337. 383.]
 [338. 354.]
 [337. 323.]
 [313. 305.]
 [296. 287.]
 [279. 285.]
 [260. 299.]
 [280. 296.]
 [297. 297.]
 [160. 301.]
 [176. 284.]
 [197. 288.]
 [217. 295.]
 [199. 300.]
 [177. 299.]
 [175. 323.]
 [191. 310.]
 [207. 317.]
 [192. 327.]
 [190. 318.]
 [302. 322.]
 [284. 312.]
 [268. 325.]
 [284. 326.]
 [282. 319.]
 [227. 319.]
 [224. 347.]
 [208. 366.]
 [215. 375.]
 [238. 380.]
 [260. 376.]
 [269. 371.]
 [251. 344.]
 [248. 317.]
 [223. 372.]
 [253. 373.]
 [199. 413.]
 [219. 399.]
 [230. 398.]
 [239. 400.]
 [247. 396.]
 [259. 399.]
 [279. 413.]
 [266. 423.]
 [254. 428.]
 [239. 428.]
 [224. 427.]
 [210. 424.]
 [222. 416.]
 [239. 417.]
 [255. 417.]
 [254. 408.]
 [240. 406.]
 [222. 408.]
 [238. 409.]
 [237. 362.]
 [182. 316.]
 [199. 318.]
 [201. 324.]
 [183. 323.]
 [290. 316.]
 [276. 316.]
 [274. 322.]
 [292. 324.]]


In [29]:
fig2 = pyplot.figure(figsize=(6, 3))
axis = fig2.add_subplot(1, 2, 1, xticks=[], yticks=[])
oimg=XR.reshape(Irows,Icols)
axis.imshow(oimg, cmap='gray')
axis.scatter(nop[0::2], nop[1::2], marker='x', s=10)

In [ ]:
pyplot.show()